#  Purification d'un gaz dans une colonne à bulle

On purifie un gaz en faisant réagir une de ces impuretés avec un réactif en solution aqueuse dans une colonne à bulles. Lorsque le gaz est introduit dans la colonne de 1 m3 on constate une augmentation de volume de 30 l. Le diamètre moyen des bulles est de 2 mm. 

**Calculer la surface de l’interface gaz-liquide dans la colonne**

>Il est possible de calculer le nombre de bulles en divisant le volume de gaz introduit 30L par le volume d'une bulle. On peut ensuite multiplier le nombre de bulles par la surface d'une bulle. 
> Plus simplement, on peut considérer la surface spécifique de sphère, 3/R (en m2/m3). La surface de l'interface est alors la surface spécifique fois le volume de gaz. 

In [1]:
import numpy as np
V_bulles=30e-3 #m3
R_bulle=1e-3 #m
V_bulle=4*np.pi*R_bulle**3/3
print (V_bulle)
nb_bulles=V_bulles/V_bulle
print (nb_bulles)
S=nb_bulles*4*np.pi*R_bulle**2
print ('Surface de l\'interface',round(S,2),'m2')
# plus simple à partir de la surface spécifique en 2 lignes
a=3/R_bulle
S2=a*V_bulles
print ('Surface de l\'interface',round(S2,2),'m2')

4.188790204786391e-09
7161972.43913529
Surface de l'interface 90.0 m2
Surface de l'interface 90.0 m2


**Calculer la vitesse ascensionnelle des bulles dans la colonne (voir rappel de mécanique des fluides).**

>L’application de la loi de Stokes peut permettre d’estimer la vitesse ascensionnelle des bulles 
$$u=\frac{2R^2}{9\mu}(\rho_L-\rho_G)g$$ si l'écoulement est laminaire Re<1
Si on calcule la vitesse avec cette relation, on trouve une vitesse d'ascension de 2,17 m/s soit un nombre de Re égale à 4350. La loi de Stokes n’est pas valide pour estimer la vitesse des bulles. Le résultat est donc FAUX.
>
>On doit alors considérer les équations donnant la force par l’intermédiaire du facteur de friction en écoulement turbulent :
>$$F_{gravité}-F_{Archimède}+F_{trainée}=0$$
>$$V_b(\rho_G-\rho_L)g+\frac{1}{2}\rho_Lu^2\pi R^2f=0$$
>
>avec un facteur de friction $f$ qui s'écrit :
>- $f=\frac{24}{Re}$ si Re<1 (équation de Stokes)
>
>- $f=\frac{18.5}{Re^{3/5}}$ si 1<Re<500 (équation de Blasius)
>
>- $f=0.44$ si Re>500
>
>Si on applique f=0.44, on trouve u=0,24 m/s et un Re=243, ce qui respecte par Re>500 et est donc FAUX.
>
>En appliquant la formule de Balsius 1<Re<500 on obtient u=0,233 m/s et Re=466, ce qui respecte l'hypothèse. C'est (enfin) le bon résultat.

In [4]:
from scipy.optimize import newton
g=9.81
rho_g=1
rho_l=1000
mu=0.10
#la fonction à résoudre pour trouver la vitesse 
def f(u):
    Re=rho_l*u*2*R_bulle/mu
    if Re<1:
        fr=24/Re
    elif Re>500:
        fr=0.44
    else:
        fr=18.5*Re**(-3/5)  
    f=V_bulle*(rho_g-rho_l)*g+0.5*rho_l*u**2*np.pi*R_bulle**2*fr
    return f
u=newton(f,x0=1)
Re=rho_l*u*2*R_bulle/mu
print ('La vitesse ascentionnelle de la bulle est de', round(u,2), 'm/s et le nombre de Reynolds est de', round(Re,2))

La vitesse ascentionnelle de la bulle est de 0.02 m/s et le nombre de Reynolds est de 0.44



Le coefficient de diffusion de l’impureté est de 1,5.10-9 m2.s-1 et la réaction en phase liquide peut être considérée comme une réaction du premier ordre de 0,15 s-1. 
	
**Estimer le coefficient de transfert de matière en utilisant la relation de Frössling :**
$$Sh=0.552Re^{0.5}Sc^{0.33}$$

In [17]:
D=1.5e-9 #m2/s
Sc=mu/(rho_l*D)
Sh=0.552*Re**0.5*Sc**0.33
k=Sh*D/(2*R_bulle)
print('le coefficient de transfert de matière est de', "%.2e"%k, 'm/s')

le coefficient de transfert de matière est de 7.74e-05 m/s


**Vérifier que le procédé utilisé est adapté en calculant le nombre de Hatta.**

>Le nombre de Hatta est alors de 0,04. La valeur de ce nombre indique que la réaction est relativement lente et constitue l’étape limitante du procédé. Il est donc inutile d’avoir une aire interfaciale élevée et la colonne à bulle qui présente une rétention liquide importante est un bon procédé à utiliser. 

In [18]:
kR=0.15
Ha=np.sqrt(kR*D/(k**2))
print ('Hatta=',round(Ha,3))

Hatta= 0.194


**Comment pourrait être modifié le procédé si jamais un réactif réagissant 100 fois plus vite avec l’impureté était mis au point ?**



In [19]:
kR_new=kR*100.
Ha_new=np.sqrt(kR_new*D/(k**2))
print ('Hatta new=',round(Ha_new,3))

Hatta new= 1.937


>Il serait alors judicieux de travailler avec une colonne à bulles agitée de façon à augmenter l’aire interfaciale.

**Calculer la densité de flux de matière dans les deux cas en considérant que l’impureté est présente à une concentration de 10-3 mole/l. **

In [20]:
def EA(Ha):
    if Ha<0.3:
        phi_f=a*D/k
        if Ha**2<phi_f:
            EA=Ha**2/phi_f
        else:
            EA=1
    elif Ha>3:
        EA=Ha
    else:
        EA=np.sqrt(1+Ha**2)
    return EA


ci=1e-3 #mol/m3
N=EA(Ha)*k*ci
N_new=EA(Ha_new)*k*ci

print ('Ha:', round(Ha,3),'le facteur d accélération est de', round(EA(Ha),3),'et le flux de matière  de', "%.2e"%N, 'mol/m2s')
print ('Ha:', round(Ha_new,3),'le facteur d accélération est de', round(EA(Ha_new),3),'et le flux de matière  de', "%.2e"%N_new, 'mol/m2s')
    

Ha: 0.194 le facteur d accélération est de 0.646 et le flux de matière  de 5.00e-08 mol/m2s
Ha: 1.937 le facteur d accélération est de 2.18 et le flux de matière  de 1.69e-07 mol/m2s


**Déterminer ensuite le temps de séjour (et la hauteur de la colonne) nécessaire pour éliminer 50% de l’impuretés. Commenter les résultats.**

>Le temps de séjour nécessaire à l’élimination peut se déterminer en effectuant un bilan sur la bulle. On considère que l'élimination de l'impuretés dans la bulle est due au flux de matière sortant de la phase gaz et arrivant dans la phase liquide. 
>
>Accumulation    =      0     -     Flux sortant
$$dn=-N.S.dt  $$
où dn est la variation de nombre de mole dans la bulle. Si l'impureté est diluée dans la phase gaz, l'élimination de l'impuretés ne changera pas le volume de la bulle, $dV=0$. La variation du nombre de mole s'écrit alors $dn=Vdc+cdV=Vdc$. L'équation de continuité conduit à l'équation différentielle suivante pour la concentration dans la bulle :
$$V\frac{dc}{dt}=-NS=-E_A k c S$$
>La résolution de l'équation conduit à une variation de la concentration en exponentielle décroissante du temps :
$$c=Ke^{-E_A a k t }$$
En appliquant la condition initiale, $t=0\quad c=c_0$, on peut écrire :
$$\frac{c}{c_0}=e^{-E_A a k t }$$
>
>le temps nécessaire à une réduction de 50% de la concentration de l’impureté est :
>$$t=-\frac{ln(0.5)}{E_A a k}$$
	 



In [21]:
t=-np.log(0.5)/(EA(Ha)*a*k)
t_new=-np.log(0.5)/(EA(Ha_new)*a*k)
H=u*t
H_new=u*t_new

print ('Ha:', round(Ha,3),'le temps est de', round(t,3),'s et la hauteur de la colonne de', round(H,3),'m')
print ('Ha:', round(Ha_new,3),'le temps est de', round(t_new,3),'s et la hauteur de la colonne de', round(H_new,3),'m')


Ha: 0.194 le temps est de 4.621 s et la hauteur de la colonne de 1.106 m
Ha: 1.937 le temps est de 1.369 s et la hauteur de la colonne de 0.328 m


Pour en savoir plus https://onlinelibrary.wiley.com/doi/book/10.1002/0471725137https://www.lavoisier.fr/livre/industries-chimiques/genie-de-la-reaction-chimique-2e-ed/villermaux/descriptif-9782852067592 disponible à la BU en plusieurs exemplaires

Questions auxquelles vous devez être capable de répondre :
- Est-ce que la résistance peut être localisée du côté gaz ?
- Est-ce que les bulles sont vraiment sphériques ?